In [28]:
%graph_notebook_host growsmart-neptune.cluster-custom-cgogeml0cuty.eu-west-3.neptune.amazonaws.com

set host to growsmart-neptune.cluster-custom-cgogeml0cuty.eu-west-3.neptune.amazonaws.com


In [18]:
%%oc
//Execute this cell (uncomment query) to delete the full graph to load again we have to trigger the bulk loader
//
// PLEASE IF YOU USE THIS CELL TO DROP THE GRAPH LET THE REST OF US KNOW
//
//MATCH (n)
//DETACH DELETE n

In [36]:
%%oc
MATCH(n:Sensor)-[c]->(s:Event)-[c2]->(d:SensorData)
RETURN n, c, s, c2, d
LIMIT 200

In [ ]:
import python-cypher

connector = Connector(host, port)
query = """
MATCH(n:Sensor)-[c]->(s:Event)-[c2]->(d:SensorData)
RETURN n, c, s, c2, d
LIMIT 200
"""



In [37]:
%db_reset --generate-token # This command will reset the db, run at your own risk

{'status': '200 OK',
 'payload': {'token': 'd4c448fc-43d2-46c6-b85e-5b3117c0834e'}}

In [39]:
%db_reset --token d4c448fc-43d2-46c6-b85e-5b3117c0834e # This command will reset the db, run at your own risk

{'status': '200 OK'}